In [41]:
import json
import tweepy
import os
import datetime

In [61]:
class TwitterAPI():
    def __init__(self, credentials_file_path = os.path.join(os.getcwd(),'credentials/twitter_credentials.json')):
        
        credentials = self.getCredentials(credentials_file_path)
        auth = tweepy.OAuthHandler(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'])
        auth.set_access_token(credentials['ACCESS_TOKEN'], credentials['ACCESS_TOKEN_SECRET'])
        self.api = tweepy.API(auth, wait_on_rate_limit=True)

    def getCredentials(self, credentials_file_path):
        with open(credentials_file_path) as json_file:
            credentials = json.load(json_file)
        return credentials
    
    def getTweetsByCount(self, query, count, loc=None, lang='en', result_type='mixed'):
        _max_queries = 500  # arbitrarily chosen value
        
        # If Country(loc) given, search on basis of country
        if(loc):
            placeId = self.getPlaceIdByCountry(loc)
            query = '{} place:{}'.format(hashtag, placeId)

        tweets = tweet_batch = self.api.search(q=query, count=count, lang=lang, result_type=result_type, tweet_mode='extended')
        ct = 1
        while len(tweets) < count and ct < _max_queries:
            tweet_batch = self.api.search(q=query, 
                                     count=count - len(tweets),
                                     max_id=tweet_batch.max_id, lang=lang, result_type=result_type, tweet_mode='extended')
            tweets.extend(tweet_batch)
            ct += 1
        return tweets
    
    def getTweetsByDates(self, query, count, start_date, end_date, loc=None, lang='en', result_type='mixed'):
        _max_queries = 500  # arbitrarily chosen value
        
        # If Country(loc) given, search on basis of country
        if(loc):
            placeId = self.getPlaceIdByCountry(loc)
            query = '{} place:{}'.format(hashtag, placeId)

        tweets = tweet_batch = self.api.search(q=query, count=count, since=start_date, unitl=end_date, lang=lang, result_type=result_type, tweet_mode='extended')
        ct = 1
        while len(tweets) < count and ct < _max_queries:
            tweet_batch = self.api.search(q=query, 
                                     count=count - len(tweets),
                                     max_id=tweet_batch.max_id, lang=lang, since=start_date, unitl=end_date, result_type=result_type, tweet_mode='extended')
            tweets.extend(tweet_batch)
            ct += 1
        return tweets
    

#     def search(self, query, count, start_date, end_date, lang='en', geocode=None, result_type='mixed'):
#         tweets = self.api.search(q=query, count=count, since=start_date, unitl=end_date, lang=lang, geocode=geocode, result_type=result_type, tweet_mode='extended')
#         features = []
#         for tweet in tweets:
#             content = {'id':tweet.id, 'timestamp':tweet.created_at, 'verified':tweet.user.verified, 'text':tweet.full_text}
#             features.append(content)
#         return features
    
    def getPlaceIdByCountry(self, loc, granularity="country"):
        places = self.api.geo_search(query=loc, granularity="country")
        return places[0].id

In [91]:
api = TwitterAPI()

In [95]:
tweets = api.getTweetsByCount('happy', 2)

In [97]:
len(tweets)

2

In [93]:
for tweet in tweets:
    print(tweet.full_text)
    print(tweet._json)
    break

I’m no political pundit but I grew up w a dad who was a federal prosecutor &amp; he taught me a lot &amp; I’ve also sat a fair amount of poker w serious players &amp; l’ll say this: I do not think Trump is trying to ‘make his base happy’ or ‘laying the groundwork for his own network’...
{'created_at': 'Fri Nov 20 10:10:47 +0000 2020', 'id': 1329728889296355328, 'id_str': '1329728889296355328', 'full_text': 'I’m no political pundit but I grew up w a dad who was a federal prosecutor &amp; he taught me a lot &amp; I’ve also sat a fair amount of poker w serious players &amp; l’ll say this: I do not think Trump is trying to ‘make his base happy’ or ‘laying the groundwork for his own network’...', 'truncated': False, 'display_text_range': [0, 287], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_re

In [92]:
api.getFeatures(tweets, ['id', 'created_at', 'full_text'])

[{'id': 1329728889296355328,
  'created_at': 'Fri Nov 20 10:10:47 +0000 2020',
  'full_text': 'I’m no political pundit but I grew up w a dad who was a federal prosecutor &amp; he taught me a lot &amp; I’ve also sat a fair amount of poker w serious players &amp; l’ll say this: I do not think Trump is trying to ‘make his base happy’ or ‘laying the groundwork for his own network’...'},
 {'id': 1329931807048028168,
  'created_at': 'Fri Nov 20 23:37:06 +0000 2020',
  'full_text': 'Happy birthday, @JoeBiden! Today you may be 78, but in two months, you’ll be 46.'},
 {'id': 1329567488829513730,
  'created_at': 'Thu Nov 19 23:29:26 +0000 2020',
  'full_text': '.@KLoeffler disclosures show you seem to have gifted yourself private jet on public dime &amp; exploited corrupt tax loopholes to do it.\n\nIf you’re happy helping yourself to public funds, maybe take a seat before arguing against stimulus checks to help everyday people in a pandemic. https://t.co/ZlhL2DjzpB'},
 {'id': 1330052487160745990

In [65]:
start_date = datetime.datetime(2020, 1, 19, 12, 00, 00)
end_date = datetime.datetime(2020, 1, 29, 13, 00, 00)

api = TwitterAPI()

tweets = api.getTweetsByDates('covid', 10, start_date, end_date)
[(tweet.full_text, tweet.created_at) for tweet in tweets]

[('✳️ COVID-19 Case Updates\n\nAs of 20.11.2020, 06:00 PM \n\nConfirmed an additional 94 cases of COVID-19 infections in Maldives\n\n- 38 Locals \n- 56 Foreigners \n\nTotal number of cases: 12,546 https://t.co/BNToLkB1wC',
  datetime.datetime(2020, 11, 20, 16, 58, 49)),
 ('#COVID19 Update (as of 11/19/20 at 12:00 am):\n• 7,126 additional positive cases of COVID-19\n• 288,978 total cases statewide \n• 9,581 deaths statewide\n• 2,629,527 patients tested negative to date\n\nMore information: https://t.co/7pzosEXhEX',
  datetime.datetime(2020, 11, 19, 16, 51, 17)),
 ("Stay Smart. Stay Safe. @MichiganHHS reports today, Nov. 16, 2020, 12,763* new COVID-19 cases &amp; 55 deaths. For * details https://t.co/2tAAcYRyCj. Michigan's total cases are now 264,576 &amp; 8,049 deaths. Find the latest data &amp; #COVID19 news @ https://t.co/niRBexCYOy. #MaskUpMichigan https://t.co/l4HSgdDzGG",
  datetime.datetime(2020, 11, 16, 20, 39, 37)),
 ('RT @PAHealthDept: #COVID19 Update (as of 11/20/20 at 12:00 a

In [76]:
api = TwitterAPI()
def search(target, date, maxnum = 100):
    cursor = tweepy.Cursor(
        api.api.search,
        q = target,
        since = date[0],
        until = date[1],
        show_user = True)

    return cursor.items(maxnum)

In [78]:
start_date = datetime.datetime(2020, 1, 19, 12, 00, 00)
end_date = datetime.datetime(2020, 1, 29, 13, 00, 00)
list(search('covid', ('2016-05-01', '2016-05-25')))

[]